In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import MeCab
import math

In [2]:
train_df = pd.read_csv("./data/train.csv")
test_df = pd.read_csv("./data/test.csv")
train_df.head()

,index,title,content,created_at,media_id
0,17958,動画 7月19日(木)朝のウェザーニュース・お天気キャスター解説,再生時間 1分35秒2018/07/19 06:46 ウェザーニュースきょう7月19日(木)...,2018-07-19 06:50:58,34
1,30130,最先端のスキンケアは「禅」+「天然水」!?セルヴォークの究極癒しスキンケアシリーズ,セルヴォークから究極の癒しスキンケアシリーズ登場！\n\n安心安全だけじゃない、本気で効果を...,2018-07-05 16:33:00,45
2,6525,コーヒーが化学反応でものすごく巨大な柱状になるGIF画像,コーヒーが化学反応でものすごく巨大な柱状になるGIF画像,2018-07-12 08:28:11,14
3,36602,姶良市 創設目指す短大は「定員240人規模」,４月に初当選した姶良市の湯元敏浩市長が公約に掲げた公立大学法人の短期大学の創設について、市は...,2018-07-03 19:43:26,23
4,30379,また大胆ファッションのベジェリン、エメリ監督の指導と新しい仲間を絶賛,『Football London』は30日、「アーセナルのDFエクトル・ベジェリンは、ウナイ...,2018-07-31 15:01:07,51


In [3]:
tagger = MeCab.Tagger("mecabrc")

In [4]:
train_df['media_id'].describe()

count    32479.000000
mean        31.659719
std         18.286326
min          0.000000
25%         16.000000
50%         32.000000
75%         47.000000
max         63.000000
Name: media_id, dtype: float64

**start**

In [5]:
train_df.columns

Index(['index', 'title', 'content', 'created_at', 'media_id'], dtype='object')

In [6]:
train_df.dtypes

index          int64
title         object
content       object
created_at    object
media_id       int64
dtype: object

**make tfidf**

In [7]:
train_df[train_df['media_id'] == 29]

,index,title,content,created_at,media_id
30,18850,女さん「会計680円に対し1180円を出すような男は気持ち悪い」,1: 風吹けば名無し 2018/07/12(木) 15:45:05.18 ID:6vSpr...,2018-07-14 07:49:22,29
282,26435,【地球こわれる】ノルウェーの北極圏で33.5度,1: 名無しさん＠涙目です。 2018/07/21(土) 16:10:12.11 ID:I...,2018-07-23 07:49:15,29
298,14341,実は販売中止になっていたと聞いて驚くお菓子ランキング 1位カール 2位サイコロキャラメル 3...,1: 名無しさん＠涙目です。 2018/06/27(水) 23:52:37.85 ID:j...,2018-07-02 23:50:26,29
300,15485,みんなスマホを鬼みたいに見てるけど何見てるの？,191: 名無しさん＠涙目です。 2018/07/28(土) 21:01:29.32 ID...,2018-07-30 22:49:33,29
445,15522,世界各国のベストセラー車種をまとめていくで,1: 名無しさん＠おーぷん 2017/02/05(日)14:07:54 ID:gdS \n...,2018-07-13 07:49:05,29
458,35556,お前らの「 ゙」と「 ゚」は全て頂いたwwwwwwwwwww,1: 以下、？ちゃんねるからVIPがお送りします 2018/07/12(木) 21:02:...,2018-07-15 11:49:29,29
533,24161,レゴランド入場料大人5000円に大幅値下げキタ━━━━(゚∀゚)━━━━!!,1: 風吹けば名無し 2018/07/18(水) 13:37:39.26 ID:/IDXO...,2018-07-18 18:49:27,29
544,32032,トラックドライバーだけど国に対してめっちゃ腹立つことがある,1: 以下、？ちゃんねるからVIPがお送りします 2018/07/18(水) 03:44:...,2018-07-19 10:49:36,29
579,24460,アイドルが握手会を欠席する理由、「嫌だから」だった,1: 風吹けば名無し 2018/07/28(土) 19:11:09.73 ID:g3qOi...,2018-07-29 00:49:19,29
823,22846,近所の理容室「顔剃り！洗髪！耳掻き！」子供ワイ「(髪切るだけでええねん...)」,1: 風吹けば名無し 2018/05/01(火) 13:57:27.70 ID:aEi+n...,2018-07-09 18:49:32,29


into list

In [8]:
title_train = list(train_df['title'])
title_test = list(test_df['title'])
all_title = title_train +title_test
content_train = list(train_df['content'])
content_test = list(test_df['content'])
all_content = content_train +content_test

In [9]:
len(all_content)

40599

In [10]:
stop_word = pd.read_csv('stop_word.csv' , header = None)
#stop_word = stop_word.values.tolist()
stop_word = stop_word.values

In [11]:
stop_word = stop_word.flatten()
stop_word = list(stop_word)

### Restart20180905

make Bow

In [12]:
##title

import MeCab

##形態素を品詞指定して行う
def extractKeyword(text):
    tagger = MeCab.Tagger('-Ochasen')
    tagger.parse('')
    node = tagger.parseToNode(text)
    #node = tagger.parse(text)
    keywords = []
    while node:
        if node.feature.split(",")[0] == "名詞":
            keywords.append(node.surface)
        #elif node.feature.split(",")[0] == "サ変接続":
        #     keywords.append(node.surface)
        #elif node.feature.split(",")[0] == "一般":
        #     keywords.append(node.surface)
        elif node.feature.split(",")[0] == "動詞":
             keywords.append(node.surface)
        node = node.next
    return keywords


#text = ["pythonでMeCabを使って形態素解析を行う。" , 'あなたは空腹なの？' , 'お前も空腹なのか']

vocab = []
for i in range(len(all_title)):
    #vocab.append(extractKeyword(text[i]))
    L = extractKeyword(all_title[i])
    joined_text = ' '.join(L)
    vocab.append(joined_text)
    
#vocab_test = []
#for i in range(len(title_test)):
#    #vocab.append(extractKeyword(text[i]))
#    L = extractKeyword(title_test[i])
#    joined_text = ' '.join(L)
#    vocab_test.append(joined_text)


In [13]:
##特徴ベクトルに変換

from sklearn.feature_extraction.text import CountVectorizer


##train_title
stop = ['(' , ')' ,'!','?' ,'w' , '@']
count = CountVectorizer(max_features =7200 , stop_words = stop_word)
#count = CountVectorizer(stop_words = stop)
docs = np.array(vocab)

bag = count.fit_transform(docs)

##単語ごとのid
#print(count.vocabulary_)


##単語ごとのidに対する出現回数
#print(bag.toarray())


##test_title
#count = CountVectorizer()
#docs_test = np.array(vocab_test)

#bag_test = count.fit_transform(docs_test)


In [14]:
##########tfidfをBoWで留めてみる

tfid = bag.toarray()

##tfidfに変換

#from sklearn.feature_extraction.text import TfidfTransformer

#tfidf = TfidfTransformer(use_idf = True , norm='l2' , smooth_idf = True)
##少数第2位まで
#np.set_printoptions(precision = 2)

##tfidfの分散表現
#print(tfidf.fit_transform(count.fit_transform(docs)).toarray())
#tfid_coluc = tfidf.fit_transform(count.fit_transform(docs)).toarray()

In [15]:
##content



##形態素を品詞指定して行う
def extractKeyword(text):
    tagger = MeCab.Tagger('-Ochasen')
    tagger.parse('')
    node = tagger.parseToNode(text)
    #node = tagger.parse(text)
    keywords = []
    while node:
        if node.feature.split(",")[0] == "名詞":
            keywords.append(node.surface)
        #elif node.feature.split(",")[0] == "サ変接続":
        #     keywords.append(node.surface)
        #elif node.feature.split(",")[0] == "一般":
        #     keywords.append(node.surface)
        elif node.feature.split(",")[0] == "動詞":
            keywords.append(node.surface)
        node = node.next
    return keywords


#text = ["pythonでMeCabを使って形態素解析を行う。" , 'あなたは空腹なの？' , 'お前も空腹なのか']

vocab = []
for i in range(len(all_content)):
    #vocab.append(extractKeyword(text[i]))
    L = extractKeyword(all_content[i])
    joined_text = ' '.join(L)
    vocab.append(joined_text)
    
#vocab_test = []
#for i in range(len(title_test)):
#    #vocab.append(extractKeyword(text[i]))
#    L = extractKeyword(title_test[i])
#    joined_text = ' '.join(L)
#    vocab_test.append(joined_text)


In [16]:
##特徴ベクトルに変換




##train_title
stop = ['(' , ')' ,'!','?' ,'w' , '@']
count = CountVectorizer(max_features =7200 , stop_words = stop_word)
#count = CountVectorizer(stop_words = stop)
docs = np.array(vocab)

bag = count.fit_transform(docs)

##単語ごとのid
#print(count.vocabulary_)


##単語ごとのidに対する出現回数
#print(bag.toarray())


##test_title
#count = CountVectorizer()
#docs_test = np.array(vocab_test)

#bag_test = count.fit_transform(docs_test)


In [17]:


##########tfidfをBoWで留めてみる

tfid_content = bag.toarray()

##tfidfに変換

#from sklearn.feature_extraction.text import TfidfTransformer

#tfidf = TfidfTransformer(use_idf = True , norm='l2' , smooth_idf = True)
##少数第2位まで
#np.set_printoptions(precision = 2)

##tfidfの分散表現
#print(tfidf.fit_transform(count.fit_transform(docs)).toarray())
#tfid_coluc2 = tfidf.fit_transform(count.fit_transform(docs)).toarray()

In [18]:
##tfid.shape
tfid_content.shape

(40599, 7200)

In [19]:
##tfidfをtrainとtestの長さで別々にする

##title
tfid_train = tfid[0:32479]
tfid_test = tfid[32479:]


##content
tfid_ctrain = tfid_content[0:32479]
tfid_ctest = tfid_content[32479:]


#title tfidf
#tfid__coluc_train = tfid_coluc[0:32479]
#tfid__coluc_test = tfid_coluc[32479:]


##content tfidf 
#tfid__coluc2_train = tfid_coluc2[0:32479]
#tfid__coluc2_test = tfid_coluc2[32479:]


#### 日付とmedia_idの分布を見る

In [20]:
train_df.columns

Index(['index', 'title', 'content', 'created_at', 'media_id'], dtype='object')

In [21]:
df = train_df[['created_at' , 'media_id']]

In [ ]:
df['created_at'].dtypes

In [ ]:
df.head(2)

In [ ]:
df['created_at'] = pd.to_datetime(df['created_at'] , format='%Y-%m-%d %H:%M:%S')

In [ ]:
df['created_at'].describe()

In [ ]:
df['week'] = df['created_at'].dt.dayofweek
df['hour'] = df['created_at'].dt.hour
#df['week_num'] = df['created_at'].dt.isocalender()

In [ ]:
#import matplotlib.pyplot as plt

sns.factorplot(x='week', y='media_id', data=df )
sns.factorplot(x='hour', y='media_id', data=df )

#### 曜日、時間帯を変数に入れても良さそう

In [20]:
##trainとtestの日付、曜日処理

train_df['created_at'] = pd.to_datetime(train_df['created_at'] , format='%Y-%m-%d %H:%M:%S')
test_df['created_at'] = pd.to_datetime(test_df['created_at'] , format='%Y-%m-%d %H:%M:%S')

train_df['week'] = train_df['created_at'].dt.dayofweek
train_df['hour'] = train_df['created_at'].dt.hour

test_df['week'] = test_df['created_at'].dt.dayofweek
test_df['hour'] = test_df['created_at'].dt.hour

train_df['mini'] = train_df['created_at'].dt.minute
test_df['mini'] = test_df['created_at'].dt.minute

In [21]:
def weekend(x):
    if ((x.week ==5)|(x.week ==6)):
        return 1
    else:
        return 0
    
train_df['weekend'] = train_df.apply(lambda x:weekend(x) , axis=1)
test_df['weekend'] = test_df.apply(lambda x:weekend(x) , axis=1)

#### trainとtestにtfidfをconcat

In [22]:
print(train_df.columns)
print(test_df.columns)

Index(['index', 'title', 'content', 'created_at', 'media_id', 'week', 'hour',
       'mini', 'weekend'],
      dtype='object')
Index(['index', 'title', 'content', 'created_at', 'week', 'hour', 'mini',
       'weekend'],
      dtype='object')


In [23]:
x_train = train_df.drop('media_id' , axis=1)
y_train = train_df['media_id']
x_test = test_df

In [24]:
####メモリ解放しておく
import sys

print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_") and sys.getsizeof(eval(var_name)) > 10000: #ここだけアレンジ
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

|            Variable Name|    Memory|
 ------------------------------------ 
|              all_content|    324856|
|                all_title|    324856|
|             content_test|     73192|
|            content_train|    292416|
|                     docs|  48069312|
|                  test_df|   5397229|
|                     tfid|2338502512|
|             tfid_content|2338502512|
|               title_test|     73192|
|              title_train|    292416|
|                 train_df|  21832426|
|                    vocab|    361288|
|                   x_test|   5397229|
|                  x_train|  21572594|
|                  y_train|    259936|


In [25]:
##delete

del tfid , docs , all_content , all_title  , content_train , content_test 

In [26]:
tfid_train_pd = pd.DataFrame(tfid_train)
tfid_test_pd = pd.DataFrame(tfid_test)

tfid_ctrain_pd = pd.DataFrame(tfid_ctrain)
tfid_ctest_pd = pd.DataFrame(tfid_ctest)


In [27]:
type(tfid_train_pd)

pandas.core.frame.DataFrame

In [28]:
x_train.drop(['index' ,'title' , 'content' , 'created_at']  ,axis=1 , inplace=True)
x_test.drop(['index' ,'title' , 'content' , 'created_at']  ,axis=1 , inplace=True)


In [29]:
x_train_all = pd.concat([x_train ,tfid_train_pd , tfid_ctrain_pd ] , axis=1)
x_test_all = pd.concat([x_test ,tfid_test_pd , tfid_ctest_pd ] , axis=1)

In [38]:
#x_train_all.replace(np.inf,np.nan).fillna(x_train_all.replace(0))
#x_test_all.replace(np.inf,np.nan).fillna(x_test_all.replace(0))

In [ ]:
test_df.shape

In [ ]:
x_train_all.shape

In [ ]:
x_test_all.shape

In [ ]:
y_train.describe()

In [30]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

#lgb_train = lgb.Dataset(x_train_all, y_train)

X_train, X_test, y_train, y_test = train_test_split(x_train_all, y_train)

# データセットを生成する
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

lgbm_params = {
        # 多値分類問題
        'objective': 'multiclass',
        # クラス数は 64
        'num_class': 64,
        'metric': 'multi_logloss',                                                                                                                                                                
        #'learning_rate': 0.1,                                                                               
        'feature_fraction': 0.9,                                                                             
        #'bagging_fraction': 0.9,                                                                             
        #'bagging_freq': 5,   
        #'lambda_l2': 0.9,
            }
    
model = lgb.train(lgbm_params, lgb_train,
                      # モデルの評価用データを渡す
                      valid_sets=lgb_eval,
                      # 最大ラウンドまで学習する
                      num_boost_round=1000,
                      # 10 ラウンド経過しても性能が向上しないときは学習を打ち切る
                      early_stopping_rounds=10,
                      #カテゴリカル変数の指定
                      categorical_feature = ['week', 'hour','weekend','mini'] ) 

y_pred = model.predict(x_test_all, num_iteration=model.best_iteration)
y_pred_max = np.argmax(y_pred, axis=1) 

/usr/lib/pyenv/versions/3.6.4/lib/python3.6/site-packages/lightgbm/basic.py:1042: UserWarning: categorical_feature in Dataset is overridden. New categorical_feature is ['hour', 'mini', 'week', 'weekend']
  warnings.warn('categorical_feature in Dataset is overridden. New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/usr/lib/pyenv/versions/3.6.4/lib/python3.6/site-packages/lightgbm/basic.py:685: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[1]	valid_0's multi_logloss: 2.45996
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's multi_logloss: 2.20505
[3]	valid_0's multi_logloss: 2.02064
[4]	valid_0's multi_logloss: 1.87325
[5]	valid_0's multi_logloss: 1.7497
[6]	valid_0's multi_logloss: 1.64296
[7]	valid_0's multi_logloss: 1.55067
[8]	valid_0's multi_logloss: 1.46867
[9]	valid_0's multi_logloss: 1.39603
[10]	valid_0's multi_logloss: 1.33012
[11]	valid_0's multi_logloss: 1.27123
[12]	valid_0's multi_logloss: 1.21803
[13]	valid_0's multi_logloss: 1.16843
[14]	valid_0's multi_logloss: 1.12306
[15]	valid_0's multi_logloss: 1.08182
[16]	valid_0's multi_logloss: 1.04415
[17]	valid_0's multi_logloss: 1.00887
[18]	valid_0's multi_logloss: 0.976242
[19]	valid_0's multi_logloss: 0.945548
[20]	valid_0's multi_logloss: 0.917586
[21]	valid_0's multi_logloss: 0.892113
[22]	valid_0's multi_logloss: 0.867702
[23]	valid_0's multi_logloss: 0.844921
[24]	valid_0's multi_logloss: 0.823228
[25]	valid_0's multi_logloss:

In [31]:
len(y_pred_max)

8120

In [32]:
test_df2 = pd.read_csv("./data/test.csv")
test_df2['media_id'] = y_pred_max

submit = test_df2[['index' ,'media_id' ]]
submit.to_csv('data/submit.csv' ,index = False)

In [ ]:
##best fit
#'learning_rate': 0.1,                                                                               
        'feature_fraction': 0.9,                                                                             
        #'bagging_fraction': 0.8,                                                                             
        #'bagging_freq': 5,   
        #'lambda_l2': 2,